In [21]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch_geometric.data import Data, DataLoader
from torch_geometric.nn import GCNConv
from torch_geometric.datasets import Planetoid
from torch_geometric.transforms import NormalizeFeatures

print("KAVYA SHRIDHAR LOLLA S20210010117, JAI SRI AISHWARYA S20210010070")
# Load the Cora dataset
dataset = Planetoid(root='data/Cora', name='Cora', transform=NormalizeFeatures())
data = dataset[0]

# Define a larger Graph Neural Network model
class LargerGNN(nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_layers, dropout=0.5):
        super(LargerGNN, self).__init__()
        self.conv_layers = nn.ModuleList()

        # First Graph Convolutional Layer
        self.conv_layers.append(GCNConv(in_channels, hidden_channels))

        # Intermediate Graph Convolutional Layers
        for _ in range(num_layers - 2):
            self.conv_layers.append(GCNConv(hidden_channels, hidden_channels))

        # Last Graph Convolutional Layer
        self.conv_layers.append(GCNConv(hidden_channels, out_channels))

        # Dropout layer for regularization
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, edge_index):
        for conv in self.conv_layers[:-1]:
            x = torch.relu(conv(x, edge_index))
            x = self.dropout(x)  # Apply dropout for regularization

        x = self.conv_layers[-1](x, edge_index)
        return x

# Instantiate the larger model
larger_model = LargerGNN(in_channels=dataset.num_node_features,
                         hidden_channels=256,  # Increased hidden layer dimensions
                         out_channels=dataset.num_classes,
                         num_layers=6)  # Increased number of layers

# Use the Adam optimizer with a larger learning rate
optimizer = optim.Adam(larger_model.parameters(), lr=0.01)

# Loss function
criterion = nn.CrossEntropyLoss()

# Training function
def train_larger_model(model, data, optimizer, criterion, epochs=200):
    model.train()
    for epoch in range(epochs):
        optimizer.zero_grad()
        out = model(data.x, data.edge_index)
        loss = criterion(out[data.train_mask], data.y[data.train_mask])
        loss.backward()
        optimizer.step()
        if epoch % 20 == 0:
            print(f'Epoch {epoch}, Loss: {loss.item()}')

# print("KAVYA SHRIDHAR LOLLA S20210010117, JAI SRI AISHWARYA S20210010070")
%prun -s cumulative -T profile_output.txt train_larger_model(larger_model, data, optimizer, criterion)


KAVYA SHRIDHAR LOLLA S20210010117, JAI SRI AISHWARYA S20210010070
Epoch 0, Loss: 1.9456897974014282
Epoch 20, Loss: 0.25219616293907166
Epoch 40, Loss: 0.03076844848692417
Epoch 60, Loss: 0.2281217724084854
Epoch 80, Loss: 0.048897407948970795
Epoch 100, Loss: 0.001608305494301021
Epoch 120, Loss: 0.017081458121538162
Epoch 140, Loss: 0.02986166812479496
Epoch 160, Loss: 0.00011302893835818395
Epoch 180, Loss: 0.22326524555683136
 
*** Profile printout saved to text file 'profile_output.txt'.


         286678 function calls (276637 primitive calls) in 28.973 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000   28.973   28.973 {built-in method builtins.exec}
        1    0.004    0.004   28.972   28.972 <string>:1(<module>)
        1    0.030    0.030   28.969   28.969 2322309956.py:54(train_larger_model)
 5000/400    0.023    0.000   14.026    0.035 module.py:1514(_wrapped_call_impl)
 5000/400    0.047    0.000   14.025    0.035 module.py:1520(_call_impl)
      200    0.034    0.000   13.998    0.070 2322309956.py:33(forward)
      200    0.002    0.000   13.833    0.069 _tensor.py:433(backward)
      200    0.003    0.000   13.830    0.069 __init__.py:149(backward)
      200   13.822    0.069   13.822    0.069 {method 'run_backward' of 'torch._C._EngineBase' objects}
     1200    0.573    0.000   11.899    0.010 gcn_conv.py:208(forward)
     1200    0.065    0.000    7.079    0.006 mes